# validate results

## TODO
what happened to tracing when I fixed probability calcs - it seems like all trip ids are attached when I add one by
hand below - why?

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import os
import argparse
from datetime import datetime

import numpy as np
import pandas as pd

from activitysim.cli import run
from activitysim.core import inject

In [4]:
pd.set_option("max_columns", 500)

In [35]:
np.argmax(np.array([[1,2,3],[4,6,5],[7,8,9]]), axis=1)

array([2, 1, 2])

In [34]:
df_ = pd.DataFrame([[1,2,3],[4,6,5],[7,8,9]], columns=["a", "b", "c"])
display(df_)
ch_ = df_.idxmax(1)
display(ch_)

,a,b,c
0,1,2,3
1,4,6,5
2,7,8,9


0    c
1    b
2    c
dtype: object

In [58]:
df_ = pd.DataFrame([[1,2,3],[4,6,5],[7,8,9]], columns=["a", "b", "c"])
display(df_)

,a,b,c
0,1,2,3
1,4,6,5
2,7,8,9


In [64]:
{v: k for k,v in enumerate(df_.columns)}

{'a': 0, 'b': 1, 'c': 2}

In [5]:
root_dir = "/mnt/c/Users/jan.zill/code/activitysim"
example_dir = os.path.join(root_dir, "test_example_mtc")

In [6]:
os.chdir(example_dir)

In [7]:
parser = argparse.ArgumentParser()
run.add_run_args(parser)
args = parser.parse_args(['-c', 'configs', '-o', 'output', '-d', 'data'])
#run.run(args)  # 2mins full example run
if not inject.is_injectable('preload_injectables'):
    from activitysim import abm  # register abm steps and other abm-specific injectables
run.handle_standard_args(args)  # possibly update injectables

In [56]:
from activitysim.core import inject
from activitysim.core import pipeline
from activitysim.core import config
from activitysim.core import simulate
from activitysim.abm.models.util import estimation
from activitysim.abm.tables import shadow_pricing
from activitysim.core import interaction_simulate
from activitysim.core import logit
from activitysim.core.simulate import set_skim_wrapper_targets
from activitysim.core import chunk

In [64]:
def hack_make_sample_choices(
        choosers, probs,
        alternatives,
        sample_size, alternative_count, alt_col_name,
        allow_zero_probs,
        trace_label):
    assert isinstance(probs, pd.DataFrame)
    assert probs.shape == (len(choosers), alternative_count)

    assert isinstance(alternatives, pd.DataFrame)
    assert len(alternatives) == alternative_count

    if allow_zero_probs:
        zero_probs = (probs.sum(axis=1) == 0)
        if zero_probs.all():
            return pd.DataFrame(columns=[alt_col_name, 'rand', 'prob', choosers.index.name])
        if zero_probs.any():
            # remove from sample
            probs = probs[~zero_probs]
            choosers = choosers[~zero_probs]

    cum_probs_array = probs.values.cumsum(axis=1)

    # alt probs in convenient layout to return prob of chose alternative
    # (same layout as cum_probs_arr)
    alt_probs_array = probs.values.flatten()

    # get sample_size rands for each chooser
    rands = pipeline.get_rn_generator().random_for_df(probs, n=sample_size)

    # transform as we iterate over alternatives
    # reshape so rands[i] is in broadcastable (2-D) shape for cum_probs_arr
    # i.e rands[i] is a 2-D array of one alt choice rand for each chooser
    rands = rands.T.reshape(sample_size, -1, 1)

    # the alternative value chosen
    choices_array = np.empty([sample_size, len(choosers)]).astype(alternatives.index.dtype)
    # chunk log these later after we populate them...
    # the probability of the chosen alternative
    choice_probs_array = np.empty([sample_size, len(choosers)])
    # chunk log these later after we populate them...
    alts = np.tile(alternatives.index.values, len(choosers))

    # FIXME - do this all at once rather than iterate?
    for i in range(sample_size):
        # FIXME - do this in numpy, not pandas?
        # rands for this alt in broadcastable shape
        r = rands[i]

        # position of first occurrence of positive value
        positions = np.argmax(cum_probs_array > r, axis=1)

        # FIXME - leave positions as numpy array, not pandas series?
        # positions is series with the chosen alternative represented as a column index in probs
        # which is an integer between zero and num alternatives in the alternative sample
        positions = pd.Series(positions, index=probs.index)

        # need to get from an integer offset into the alternative sample to the alternative index
        # that is, we want the index value of the row that is offset by <position> rows into the
        # tranche of this choosers alternatives created by cross join of alternatives and choosers

        # offsets is the offset into model_design df of first row of chooser alternatives
        offsets = np.arange(len(positions)) * alternative_count

        # choices and choice_probs have one element per chooser and is in same order as choosers
        choices_array[i] = np.take(alts, positions + offsets)
        choice_probs_array[i] = np.take(alt_probs_array, positions + offsets)

        del positions
        del offsets

    del alts
    del cum_probs_array
    del alt_probs_array

    # explode to one row per chooser.index, alt_zone_id
    choices_df = pd.DataFrame(
        {alt_col_name: choices_array.flatten(order='F'),
         'rand': rands.flatten(order='F'),
         'prob': choice_probs_array.flatten(order='F'),
         choosers.index.name: np.repeat(np.asanyarray(choosers.index), sample_size)
         })

    del choices_array
    del rands
    del choice_probs_array
    return choices_df

In [92]:
%%time
resume_after = "compute_accessibility"
model_name = "school_location"
chunk_size = 0  # test_mtc means no chunking

pipeline.open_pipeline(resume_after)
# preload any bulky injectables (e.g. skims) not in pipeline
inject.get_injectable('preload_injectables', None)
pipeline._PIPELINE.rng().begin_step(model_name)
#step_name = model_name
args = {}
#checkpoint = pipeline.intermediate_checkpoint(model_name)
inject.set_step_args(args)


persons_merged = inject.get_table('persons_merged')
network_los = inject.get_injectable('network_los')
households = inject.get_table('households')
persons = inject.get_table('persons')
locutor = inject.get_injectable('locutor')

trace_label = model_name #'school_location'
model_settings_file_name = f"{model_name}.yaml" #'school_location.yaml'
model_settings = config.read_model_settings(model_settings_file_name)

estimator = estimation.manager.begin_estimation(model_name)

# iterate_location_choice(
#     model_settings,
#     persons_merged, persons, households,
#     network_los,
#     estimator,
#     chunk_size, trace_hh_id, locutor, trace_label
# )

chunk_tag = trace_label

# boolean to filter out persons not needing location modeling (e.g. is_worker, is_student)
chooser_filter_column = model_settings['CHOOSER_FILTER_COLUMN_NAME']

dest_choice_column_name = model_settings['DEST_CHOICE_COLUMN_NAME']
logsum_column_name = model_settings.get('DEST_CHOICE_LOGSUM_COLUMN_NAME')

sample_table_name = model_settings.get('DEST_CHOICE_SAMPLE_TABLE_NAME')
want_sample_table = config.setting('want_dest_choice_sample_tables') and sample_table_name is not None

persons_merged_df = persons_merged.to_frame()

persons_merged_df = persons_merged_df[persons_merged_df[chooser_filter_column]]

persons_merged_df.sort_index(inplace=True)  # interaction_sample expects chooser index to be monotonic increasing

# chooser segmentation allows different sets coefficients for e.g. different income_segments or tour_types
chooser_segment_column = model_settings['CHOOSER_SEGMENT_COLUMN_NAME']

assert chooser_segment_column in persons_merged_df, f"CHOOSER_SEGMENT_COLUMN '{chooser_segment_column}' not in " \
                                                    f"persons_merged table."

shadow_price_calculator = shadow_pricing.load_shadow_price_calculator(model_settings)

chooser_segment_column = model_settings['CHOOSER_SEGMENT_COLUMN_NAME']

# maps segment names to compact (integer) ids
segment_ids = model_settings['SEGMENT_IDS']

#choices_list = []
sample_list = []
for segment_name, segment_id in segment_ids.items():
    print(f"running {segment_name}, {segment_id}")
    choosers = persons_merged_df[persons_merged_df[chooser_segment_column] == segment_id]

    # size_term and shadow price adjustment - one row per zone
    dest_size_terms = shadow_price_calculator.dest_size_terms(segment_name)

    assert dest_size_terms.index.is_monotonic_increasing, f"shadow_price_calculator.dest_size_terms({segment_name}) " \
                                                         f"not monotonic_increasing"
    if choosers.shape[0] == 0:
        print(f"{trace_label} skipping segment {segment_name}: no choosers")
        continue

    print(f"dropping {(~(dest_size_terms.size_term > 0)).sum()} "
          f"of {len(dest_size_terms)} rows where size_term is zero")
    dest_size_terms = dest_size_terms[dest_size_terms.size_term > 0]

    chooser_columns = model_settings['SIMULATE_CHOOSER_COLUMNS']
    choosers_location_sample = choosers[chooser_columns]
    skim_dict = network_los.get_default_skim_dict()
    skims = skim_dict.wrap('home_zone_id', 'zone_id')
    alt_dest_col_name = model_settings['ALT_DEST_COL_NAME']

    assert not choosers_location_sample.empty
    print("Running %s with %d persons" % (trace_label, len(choosers_location_sample.index)))
    sample_size = model_settings["SAMPLE_SIZE"]
    locals_d = {
        'skims': skims,
        'segment_size': segment_name
    }
    constants = config.get_model_constants(model_settings)
    locals_d.update(constants)
    spec = simulate.spec_for_segment(model_settings, spec_id='SAMPLE_SPEC',
                                     segment_name=segment_name, estimator=estimator)
    ### choices = interaction_sample(
    alt_col_name=alt_dest_col_name
    allow_zero_probs=False
    log_alt_losers=False

    # we return alternatives ordered in (index, alt_col_name)
    # if choosers index is not ordered, it is probably a mistake, since the alts wont line up
    assert alt_col_name is not None
    assert choosers.index.is_monotonic_increasing

    # FIXME - legacy logic - not sure this is needed or even correct?
    sample_size = min(sample_size, len(dest_size_terms.index))

    result_list = []
    for i, chooser_chunk, chunk_trace_label in chunk.adaptive_chunked_choosers(choosers_location_sample, chunk_size, trace_label,
                                                                               chunk_tag):

        ### choices = hack_interaction_sample
        # chooser = chooser_chunk
        #alternatives = dest_size_terms
        #trace_label=chunk_trace_label

        num_choosers = len(chooser_chunk.index)
        assert num_choosers > 0
        if len(spec.columns) > 1:
            raise RuntimeError('spec must have only one column')
        # if using skims, copy index into the dataframe, so it will be
        # available as the "destination" for set_skim_wrapper_targets
        if skims is not None and dest_size_terms.index.name not in dest_size_terms:
            dest_size_terms = dest_size_terms.copy()
            dest_size_terms[dest_size_terms.index.name] = dest_size_terms.index

        chooser_index_id = interaction_simulate.ALT_CHOOSER_ID if log_alt_losers else None

        # - cross join choosers and alternatives (cartesian product)
        # for every chooser, there will be a row for each alternative
        # index values (non-unique) are from alternatives df
        alternative_count = dest_size_terms.shape[0]
        interaction_df =\
            logit.interaction_dataset(chooser_chunk, dest_size_terms, sample_size=alternative_count,
                                      chooser_index_id=chooser_index_id)

        assert alternative_count == len(interaction_df.index) / len(chooser_chunk.index)

        if skims is not None:
            set_skim_wrapper_targets(interaction_df, skims)

        # evaluate expressions from the spec multiply by coefficients and sum
        # spec is df with one row per spec expression and one col with utility coefficient
        # column names of interaction_df match spec index values
        # utilities has utility value for element in the cross product of choosers and alternatives
        # interaction_utilities is a df with one utility column and one row per row in interaction_df
        trace_rows = trace_ids = None

        # interaction_utilities is a df with one utility column and one row per interaction_df row
        interaction_utilities, trace_eval_results = interaction_simulate.eval_interaction_utilities(
            spec, interaction_df, locals_d, chunk_trace_label, trace_rows, estimator=None,
            log_alt_losers=log_alt_losers
        )
        # ########### HWM - high water mark (point of max observed memory usage)
        #del interaction_df

        # reshape utilities (one utility column and one row per row in interaction_utilities)
        # to a dataframe with one row per chooser and one column per alternative
        utilities = pd.DataFrame(
            interaction_utilities.values.reshape(len(chooser_chunk), alternative_count),
            index=chooser_chunk.index)
        #del interaction_utilities

        # convert to probabilities (utilities exponentiated and normalized to probs)
        # probs is same shape as utilities, one row per chooser and one column for alternative
        probs = logit.utils_to_probs(utilities, allow_zero_probs=allow_zero_probs,
                                     trace_label=chunk_trace_label, trace_choosers=chooser_chunk)
        #del utilities

        temp_choices = hack_make_sample_choices(
            chooser_chunk, probs, dest_size_terms,
            sample_size, alternative_count, alt_col_name,
            allow_zero_probs=allow_zero_probs,
            trace_label=chunk_trace_label)

        choices_df = temp_choices.copy()

        # pick_count and pick_dup
        # pick_count is number of duplicate picks
        # pick_dup flag is True for all but first of duplicates
        pick_group = choices_df.groupby([choosers.index.name, alt_col_name])
        # number each item in each group from 0 to the length of that group - 1.
        choices_df['pick_count'] = pick_group.cumcount(ascending=True)
        # flag duplicate rows after first
        choices_df['pick_dup'] = choices_df['pick_count'] > 0
        # add reverse cumcount to get total pick_count (conveniently faster than groupby.count + merge)
        choices_df['pick_count'] += pick_group.cumcount(ascending=False) + 1
        # drop the duplicates
        choices_df = choices_df[~choices_df['pick_dup']]
        del choices_df['pick_dup']
        # set index after groupby so we can trace on it
        choices_df.set_index(choosers.index.name, inplace=True)
        # don't need this after tracing
        del choices_df['rand']
        # - NARROW
        choices_df['prob'] = choices_df['prob'].astype(np.float32)
        assert (choices_df['pick_count'].max() < 4294967295) or (choices_df.empty)
        choices_df['pick_count'] = choices_df['pick_count'].astype(np.uint32)


        if choices_df.shape[0] > 0:
            result_list.append(choices_df)

    if len(result_list) > 1:
        choices_df = pd.concat(result_list)
    assert allow_zero_probs or (len(choosers_location_sample.index) == len(np.unique(choices_df.index.values))), \
        "what is this"
    # keep alts in canonical order so choices based on their probs are stable across runs
    choices_df = choices_df.sort_values(by=alt_col_name).sort_index(kind='mergesort')

    sample_list.append(choices_df)

finalise = True
if finalise:
    inject.set_step_args(None)
    #
    pipeline._PIPELINE.rng().end_step(model_name)
    pipeline.add_checkpoint(model_name)
    if not pipeline.intermediate_checkpoint():
        pipeline.add_checkpoint(pipeline.FINAL_CHECKPOINT_NAME)

    pipeline.close_pipeline()

estimation bundle school_location not in settings file estimation.yaml


running university, 3
dropping 19 of 25 rows where size_term is zero
Running school_location with 17 persons
running highschool, 2
dropping 23 of 25 rows where size_term is zero
Running school_location with 5 persons
running gradeschool, 1
dropping 0 of 25 rows where size_term is zero
Running school_location with 17 persons
CPU times: user 188 ms, sys: 31.2 ms, total: 219 ms
Wall time: 216 ms


In [98]:
temp_choices

,alt_dest,rand,prob,person_id
0,8,0.222922,0.157982,386008
1,9,0.404176,0.336708,386008
2,8,0.223154,0.157982,386008
3,9,0.457402,0.336708,386008
4,11,0.812506,0.069858,386008
...,...,...,...,...
165,7,0.107363,0.062739,4171620
166,11,0.517573,0.070848,4171620
167,10,0.375828,0.097048,4171620
168,9,0.279810,0.111409,4171620


In [97]:
sample_list[-1]

,alt_dest,prob,pick_count
person_id,,,
386008,8,0.157982,2
386008,9,0.336708,5
386008,10,0.205884,1
386008,11,0.069858,1
386008,17,0.009346,1
...,...,...,...
4171620,9,0.111409,1
4171620,10,0.097048,1
4171620,11,0.070848,1


In [91]:
display(sample_size)
sample_list[2].groupby('person_id').pick_count.sum()

10

person_id
386008     10
418442     10
595686     10
644292     10
644478     10
1958678    10
2159059    10
2219998    10
2458502    10
2458503    10
2566700    10
2566701    10
2566702    10
2877287    10
3596365    10
3891104    10
4171620    10
Name: pick_count, dtype: uint32

AssertionError: 